In [2]:
import numpy as np
import random
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import pymysql


In [3]:
# Memuat file Excel
def load_matrix(file_path):
    df = pd.read_excel(file_path, index_col=0)
    df.columns = df.columns.astype(str).str.strip()
    df.index = df.index.astype(str).str.strip()
    return df

In [4]:
# Load DataFrame (asumsi menggunakan fungsi `load_matrix()`)
file_path = "aggregated_by_department_normalized.xlsx"
markov_matrix = load_matrix(file_path)

# Pastikan tipe data DataFrame adalah float
markov_matrix = markov_matrix.astype(float)

epsilon = 1e-6  # Nilai kecil untuk normalisasi

# Perbaiki state yang tidak memiliki transisi
for index, row in markov_matrix.iterrows():
    if np.isclose(np.sum(row), 1.0):  # State absorbing tetap dipertahankan
        continue
    elif np.isclose(np.sum(row), 0.0):  # Jika tidak ada transisi, tambahkan probabilitas kecil
        markov_matrix.loc[index] = np.ones(len(row), dtype=np.float64) * epsilon
        markov_matrix.loc[index, index] = 1 - (epsilon * (len(row) - 1))  # Menjaga normalisasi

# Pastikan matriks transisi tetap valid (jumlah setiap baris harus 1)
markov_matrix = markov_matrix.div(markov_matrix.sum(axis=1), axis=0)

# Konversi DataFrame ke NumPy sebelum matrix power
markov_numpy = markov_matrix.to_numpy()

print("Baris:", len(markov_matrix), "Kolom:", len(markov_matrix.columns))

Baris: 139 Kolom: 139


In [5]:
# Nama baris yang dicari
# kurangi_depart = "Worker_Yard Operations Department_Gate Operator"
# tambah_depart= "Worker_Yard Operations Department_Tallyman Loading Discharge"


depart_kurang = {"Operations Department" : 4, "Finance & Administration Department": 2}
# quantity_kurang = [4,2]
depart_tambah = {"Production Department":2, "Surabaya Branch Office Department":2,"Treasury Department":2}
# quantity_tambah = [2,2,2]

# nanti input dan output berupa 2 array dictionary yang berisi value nama seperti Worker_Yard Operations Department_Gate Operator dan quantity (integer lebih dari 1)

if all(k in markov_matrix.index for k in depart_kurang) & all(t in markov_matrix.index for t in depart_tambah):
    output = []
    for index1, q1 in depart_kurang.items():
        new_posisi = []
        new_probabilty = []
        search_array = markov_matrix.loc[index1].to_numpy()
        mc_p1 = np.linalg.matrix_power(markov_numpy,1)
        hasil = np.dot(search_array, mc_p1)
        for index2, q2 in depart_tambah.items():
            new_posisi.append((markov_matrix.columns.get_loc(index2)))
            temp = hasil[(markov_matrix.columns.get_loc(index2))]
            new_probabilty.append((temp))
            print(f"Probabilitas menuju state '{index2}': {temp}")            
        print("--------------------------------------------------------------------")
else:
    print("error")

Probabilitas menuju state 'Production Department': 0.027998228865271424
Probabilitas menuju state 'Surabaya Branch Office Department': 2.4752475247524726e-09
Probabilitas menuju state 'Treasury Department': 1.38267246570974e-05
--------------------------------------------------------------------
Probabilitas menuju state 'Production Department': 0.007033968650889921
Probabilitas menuju state 'Surabaya Branch Office Department': 2.2522522522522443e-09
Probabilitas menuju state 'Treasury Department': 0.004857225808298242
--------------------------------------------------------------------


In [6]:

# Misal markov_matrix dan markov_numpy sudah didefinisikan sebelumnya
# Contoh dictionary input:
depart_kurang = {"Operations Department" : 4, "Finance & Administration Department": 2}
# quantity_kurang = [4,2]
depart_tambah = {"Production Department":2, "Surabaya Branch Office Department":2,"Treasury Department":2}
# quantity_tambah = [2,2,2]

# Pastikan nama row/column ada di markov_matrix
if all(k in markov_matrix.index for k in depart_kurang) and all(t in markov_matrix.index for t in depart_tambah):
    
    output = []  # Output akhir: list yang berisi list alokasi untuk tiap depart_kurang
    # Buat salinan kapasitas global untuk depart_tambah (akan dikurangi seiring alokasi)
    remaining_capacity = depart_tambah.copy()
    # Urutan key dari depart_tambah (penting agar output konsisten)
    depart_tambah_keys = list(depart_tambah.keys())
    
    # Loop tiap depart_kurang
    for index1, q1 in depart_kurang.items():
        new_probability = []
        
        # Ambil baris dari markov_matrix untuk depart_kurang
        search_array = markov_matrix.loc[index1].to_numpy()
        mc_p1 = np.linalg.matrix_power(markov_numpy, 1)
        hasil = np.dot(search_array, mc_p1)
        
        # Hitung probabilitas untuk tiap depart_tambah (sesuai urutan depart_tambah_keys)
        for key in depart_tambah_keys:
            col_index = markov_matrix.columns.get_loc(key)
            prob = hasil[col_index]
            new_probability.append(prob)
            print(f"Probabilitas menuju state '{key}': {prob}")
        print("---------------------------------------------------------------------")
        
        # Normalisasi probabilitas agar jumlahnya 1 (jika total > 0)
        total_prob = sum(new_probability)
        if total_prob > 0:
            norm_probability = [p / total_prob for p in new_probability]
        else:
            norm_probability = [1 / len(new_probability)] * len(new_probability)
        
        # Inisialisasi alokasi untuk baris ini, dalam bentuk list dengan panjang sama dengan jumlah depart_tambah
        allocation = [0] * len(depart_tambah_keys)
        items_to_allocate = q1
        
        # Lakukan alokasi item sebanyak q1 unit
        while items_to_allocate > 0:
            # Buat daftar kandidat (indeks) dari depart_tambah yang masih memiliki sisa kapasitas
            candidates = []
            candidate_weights = []
            for i, key in enumerate(depart_tambah_keys):
                if remaining_capacity[key] > 0:
                    candidates.append(i)
                    candidate_weights.append(norm_probability[i])
            
            # Jika tidak ada kandidat tersisa, keluarkan pesan peringatan dan hentikan alokasi untuk baris ini
            if not candidates:
                print("Tidak ada kapasitas tersisa untuk alokasi!")
                break
            
            # Pilih secara acak salah satu kandidat menggunakan bobot
            chosen_idx = random.choices(candidates, weights=candidate_weights, k=1)[0]
            allocation[chosen_idx] += 1
            remaining_capacity[depart_tambah_keys[chosen_idx]] -= 1
            items_to_allocate -= 1
        
        output.append(allocation)
    
    print("Distribusi akhir:")
    print(output)
else:
    print("error")

Probabilitas menuju state 'Production Department': 0.027998228865271424
Probabilitas menuju state 'Surabaya Branch Office Department': 2.4752475247524726e-09
Probabilitas menuju state 'Treasury Department': 1.38267246570974e-05
---------------------------------------------------------------------
Probabilitas menuju state 'Production Department': 0.007033968650889921
Probabilitas menuju state 'Surabaya Branch Office Department': 2.2522522522522443e-09
Probabilitas menuju state 'Treasury Department': 0.004857225808298242
---------------------------------------------------------------------
Distribusi akhir:
[[2, 0, 2], [0, 2, 0]]


level = ['Director', 'Expatriat', 'General Manager', 'Junior Manager', 'Middle Manager', 'Non Grade', 'President Director', 'Senior Manager', 'Senior Staff', 'Staff', 'Team Leader', 'Trainee', 'Worker']
department = ['Account Receivable Core Department', 'Account Receivable Department', 'Accounting Department', 'Administrasi Depo', 'Administration Department', 'BOD', 'Branch Class B East', 'Branch Class C East', 'Branches Management Department', 'Bulk Management Department', 'Bunker And Overseas Purchasing Department', 'Business Development Department', 'Car Carrier', 'Chinese Desk', 'Commercial Department', 'Commercial Jakarta Department', 'Continuous Improvement Department', 'Core Department', 'Corporate Account Department', 'Corporate Communication', 'Customer Service Department', 'DFF', 'DPA Department', 'DPL', 'Facility Electrical Department', 'Feeder, SOC & International Trade Sales Department', 'Finance & Administration Department', 'Finance Department', 'Finance and Accounting 

In [28]:

try:
    # Koneksi ke database
    connection = pymysql.connect(
        user="root",
        password="",
        host="localhost",
        port=3306,
        database="markovkaryawandb"
    )

    cursor = connection.cursor()

    # Data department dengan jumlah kekurangan pegawai
    depart_kurang = {
        "Operations Department": 4,
        "Finance & Administration Department": 2
    }

    output = []

    for name, q1 in depart_kurang.items():  # Menggunakan .items() agar bisa mengakses key dan value
        # Menggunakan parameterized query untuk menghindari SQL injection
        depart_id_query = "SELECT id FROM department WHERE name = %s"
        cursor.execute(depart_id_query, (name,))
        depart_id = cursor.fetchall()
        
        query = "SELECT id, full_name, position FROM employee WHERE department = %s"
        cursor.execute(query, (depart_id,))

        # Fetch semua data
        users = cursor.fetchall()

        # Menambahkan hasil ke output
        output.append({
            "department": name,
            "needed": q1,
            "employees": users
        })

    # Menutup koneksi setelah looping selesai
    cursor.close()
    connection.close()

    # Menampilkan hasil
    print(output)

except Exception as e:
    print({"message": str(e)})


[{'department': 'Operations Department', 'needed': 4, 'employees': ((126, 'Scott Kerr', '6'), (142, 'Erik Baker', '4'), (153, 'Erin Dixon', '9'), (312, 'Michelle Hubbard DVM', '10'), (501, 'Scott Black', '9'), (739, 'Daniel Potter', '12'), (752, 'Sarah Taylor', '8'))}, {'department': 'Finance & Administration Department', 'needed': 2, 'employees': ((132, 'Andrea Watts', '10'), (262, 'Andrew Duncan', '6'), (330, 'Karen Rowe', '6'), (550, 'Cole Singh', '4'), (583, 'Kara Fischer', '5'), (760, 'Jesse Andrews', '8'), (814, 'Peter Morgan', '7'), (868, 'Brent Harrison', '12'), (880, 'Rebecca Wright', '8'))}]


In [ ]:
PATH = "FRONT_END -> markov.py <-> database -> classification -> random_mutation_people -> FRONT_END" 

In [8]:
import pandas as pd
from IPython.display import FileLink  # Opsional, hanya untuk Jupyter Notebook

# Fungsi untuk mengekstrak bagian department dari label dengan format "level_department_job"
def extract_department(label):
    parts = label.split('_', 2)  # Maksimal 2 pemisah
    if len(parts) == 3:
        return parts[1].strip()
    else:
        return label.strip()

# Baca file Excel input
input_file = 'matrix.xlsx'  # Pastikan file input.xlsx berada di direktori yang sama
# Asumsi: Baris pertama adalah header kolom dan kolom pertama adalah index (label baris)
df = pd.read_excel(input_file, index_col=0)

print("DataFrame Asli:")
print(df)

# Ubah label baris (index) dan kolom dengan mengekstrak department
df.index = df.index.map(extract_department)
df.columns = df.columns.map(extract_department)

# Grouping baris berdasarkan department dan jumlahkan nilainya
df_grouped_rows = df.groupby(df.index).sum()

# Grouping kolom berdasarkan department (axis=1) dan jumlahkan nilainya
df_grouped = df_grouped_rows.groupby(df_grouped_rows.columns, axis=1).sum()

print("\nDataFrame setelah grouping berdasarkan department:")
print(df_grouped)

# Normalisasi tiap baris: bagi setiap nilai dengan jumlah nilai pada baris tersebut
df_normalized = df_grouped.div(df_grouped.sum(axis=1), axis=0)

print("\nDataFrame setelah normalisasi (setiap baris berjumlah 1):")
print(df_normalized)

# Simpan DataFrame hasil normalisasi ke file Excel baru
output_file = 'aggregated_by_department_normalized.xlsx'
df_normalized.to_excel(output_file)
print("\nFile output telah disimpan sebagai:", output_file)

# Jika menggunakan Jupyter Notebook, tampilkan link download (opsional)
try:
    display(FileLink(output_file))
except Exception:
    pass


KeyboardInterrupt: 

In [9]:

try:
    # Koneksi ke database
    connection = pymysql.connect(
        user="root",
        password="",
        host="localhost",
        port=3306,
        database="markovkaryawandb"
    )

    cursor = connection.cursor()

    # Data department dengan jumlah kekurangan pegawai

    # Misal markov_matrix dan markov_numpy sudah didefinisikan sebelumnya
    # Contoh dictionary input:
    depart_kurang = {"Operations Department" : 4, "Finance & Administration Department": 2}
    # quantity_kurang = [4,2]
    depart_tambah = {"Production Department":2, "Surabaya Branch Office Department":2,"Treasury Department":2}
    # quantity_tambah = [2,2,2]

    # Pastikan nama row/column ada di markov_matrix
    if all(k in markov_matrix.index for k in depart_kurang) and all(t in markov_matrix.index for t in depart_tambah):
        
        output = []  # Output akhir: list yang berisi list alokasi untuk tiap depart_kurang
        # Buat salinan kapasitas global untuk depart_tambah (akan dikurangi seiring alokasi)
        remaining_capacity = depart_tambah.copy()
        # Urutan key dari depart_tambah (penting agar output konsisten)
        depart_tambah_keys = list(depart_tambah.keys())
        
        # Loop tiap depart_kurang
        for index1, q1 in depart_kurang.items():
            new_probability = []
            
            # Ambil baris dari markov_matrix untuk depart_kurang
            search_array = markov_matrix.loc[index1].to_numpy()
            mc_p1 = np.linalg.matrix_power(markov_numpy, 1)
            hasil = np.dot(search_array, mc_p1)
            
            # Hitung probabilitas untuk tiap depart_tambah (sesuai urutan depart_tambah_keys)
            for key in depart_tambah_keys:
                col_index = markov_matrix.columns.get_loc(key)
                prob = hasil[col_index]
                new_probability.append(prob)
                print(f"Probabilitas menuju state '{key}': {prob}")
            print("---------------------------------------------------------------------")
            
            # Normalisasi probabilitas agar jumlahnya 1 (jika total > 0)
            total_prob = sum(new_probability)
            if total_prob > 0:
                norm_probability = [p / total_prob for p in new_probability]
            else:
                norm_probability = [1 / len(new_probability)] * len(new_probability)
            
            # Inisialisasi alokasi untuk baris ini, dalam bentuk list dengan panjang sama dengan jumlah depart_tambah
            allocation = [0] * len(depart_tambah_keys)
            items_to_allocate = q1
            
            # Lakukan alokasi item sebanyak q1 unit
            while items_to_allocate > 0:
                # Buat daftar kandidat (indeks) dari depart_tambah yang masih memiliki sisa kapasitas
                candidates = []
                candidate_weights = []
                for i, key in enumerate(depart_tambah_keys):
                    if remaining_capacity[key] > 0:
                        candidates.append(i)
                        candidate_weights.append(norm_probability[i])
                
                # Jika tidak ada kandidat tersisa, keluarkan pesan peringatan dan hentikan alokasi untuk baris ini
                if not candidates:
                    print("Tidak ada kapasitas tersisa untuk alokasi!")
                    break
                
                # Pilih secara acak salah satu kandidat menggunakan bobot
                chosen_idx = random.choices(candidates, weights=candidate_weights, k=1)[0]
                allocation[chosen_idx] += 1
                remaining_capacity[depart_tambah_keys[chosen_idx]] -= 1
                items_to_allocate -= 1
            
            output.append(allocation)
        
        print("Distribusi akhir:")
        print(output)
    else:
        print("error")

    output = []

    for name, q1 in depart_kurang.items():  # Menggunakan .items() agar bisa mengakses key dan value
        # Menggunakan parameterized query untuk menghindari SQL injection
        query = "SELECT id, full_name, position FROM employee WHERE department = %s"
        cursor.execute(query, (name,))

        # Fetch semua data
        users = cursor.fetchall()

        # Menambahkan hasil ke output
        output.append({
            "department": name,
            "needed": q1,
            "employees": users
        })

    # Menutup koneksi setelah looping selesai
    cursor.close()
    connection.close()

    # Menampilkan hasil
    print(output)

except Exception as e:
    print({"message": str(e)})

Probabilitas menuju state 'Production Department': 0.027998228865271424
Probabilitas menuju state 'Surabaya Branch Office Department': 2.4752475247524726e-09
Probabilitas menuju state 'Treasury Department': 1.38267246570974e-05
---------------------------------------------------------------------
Probabilitas menuju state 'Production Department': 0.007033968650889921
Probabilitas menuju state 'Surabaya Branch Office Department': 2.2522522522522443e-09
Probabilitas menuju state 'Treasury Department': 0.004857225808298242
---------------------------------------------------------------------
Distribusi akhir:
[[2, 0, 2], [0, 2, 0]]
[{'department': 'Operations Department', 'needed': 4, 'employees': ()}, {'department': 'Finance & Administration Department', 'needed': 2, 'employees': ()}]


In [32]:
!pip install ace_tools

In [19]:
file_path = "Data_Pegawai_Final.csv"
df = pd.read_csv(file_path)

print(df)

       ID  bars  kpi    State  Selisih  Position  Tahun
0      11    83   83    tetap        1         6   2021
1      11    98   78    tetap        2         6   2022
2      11    68   61    tetap        3         6   2023
3      11    76   69    tetap        4         6   2024
4      11    73   88    tetap        5         6   2025
...   ...   ...  ...      ...      ...       ...    ...
4420  895    93   96  promosi        1         6   2021
4421  895    68   88    tetap        1         5   2022
4422  895    65   72    tetap        2         5   2023
4423  895    71   63    tetap        3         5   2024
4424  895    64   87    tetap        4         5   2025

[4425 rows x 7 columns]


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [20]:


# Misalkan 'df' adalah DataFrame yang berisi data kamu
# Fitur (X) dan Target (y)
X = df[['Selisih', 'bars', 'kpi', 'Position']]  # Fitur yang digunakan untuk prediksi
y = df['State']  # Kolom 'State' sebagai target untuk prediksi

# Lakukan One-Hot Encoding pada target 'State' (mutasi, tetap, promosi)
target_encoder = OneHotEncoder(sparse_output=False)
y_encoded = target_encoder.fit_transform(y.values.reshape(-1, 1))

# Membagi data menjadi train dan test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Menampilkan ukuran data train dan test
print("Ukuran Train Set:", X_train.shape)
print("Ukuran Test Set:", X_test.shape)

Ukuran Train Set: (3540, 4)
Ukuran Test Set: (885, 4)


In [25]:
# Membuat dan melatih model Random Forest
model = RandomForestClassifier(n_estimators=1, random_state=42)
# model2 = LogisticRegression(random_state=42)
# Melatih model dengan data train
model.fit(X_train, y_train)
# model2.fit(X_train, y_train)

# Memprediksi dengan data test
y_pred = model.predict(X_test)
# y_pred2 = model2.predict(X_test)

# Menghitung akurasi model
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi: {accuracy:.4f}")
# accuracy2 = accuracy_score(y_test, y_pred2)
# print(f"Akurasi: {accuracy2:.4f}")

# Menampilkan laporan klasifikasi
print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))
# print(classification_report(y_test, y_pred2))

Akurasi: 0.9932
Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94        23
           1       1.00      0.98      0.99       153
           2       0.99      1.00      1.00       709

    accuracy                           0.99       885
   macro avg       0.97      0.98      0.97       885
weighted avg       0.99      0.99      0.99       885



In [22]:
# Misalkan 'df' adalah DataFrame yang berisi data kamu
# Fitur (X) dan Target (y)
X = df[['Selisih', 'bars', 'kpi', 'Position']]  # Fitur yang digunakan untuk prediksi
y = df['State']  # Kolom 'State' sebagai target untuk prediksi

# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Encode label 'State' menjadi angka (0: mutasi, 1: tetap, 2: promosi)
y_encoded = label_encoder.fit_transform(y)

# Membagi data menjadi train dan test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Menampilkan ukuran data train dan test
print("Ukuran Train Set:", X_train.shape)
print("Ukuran Test Set:", X_test.shape)

# Melatih model Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Memprediksi dengan data test
y_pred = model.predict(X_test)

# Menghitung akurasi model
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi: {accuracy:.4f}")

# Menampilkan laporan klasifikasizz
print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))

# Prediksi untuk input manual
input_manual = pd.DataFrame([[5, 94, 85, 5]], columns=['Selisih', 'bars', 'kpi', 'Position'])

# Prediksi dengan model yang sudah dilatih
prediksi = model.predict(input_manual)

# Mengembalikan hasil prediksi ke label yang dapat dimengerti
prediksi_label = label_encoder.inverse_transform(prediksi)

print(f"Prediksi untuk input manual: {prediksi_label[0]}")

Ukuran Train Set: (3540, 4)
Ukuran Test Set: (885, 4)
Akurasi: 1.0000
Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00       153
           2       1.00      1.00      1.00       709

    accuracy                           1.00       885
   macro avg       1.00      1.00      1.00       885
weighted avg       1.00      1.00      1.00       885

Prediksi untuk input manual: promosi


In [23]:
from sklearn.neighbors import KNeighborsClassifier
# Misalkan 'df' adalah DataFrame yang berisi data kamu
# Fitur (X) dan Target (y)
def label(selisih, bars, kpi, position):
    X = df[['Selisih', 'bars', 'kpi', 'Position']]  # Fitur yang digunakan untuk prediksi
    y = df['State']  # Kolom 'State' sebagai target untuk prediksi

    # Inisialisasi LabelEncoder
    label_encoder = LabelEncoder()

    # Encode label 'State' menjadi angka (0: mutasi, 1: tetap, 2: promosi)
    y_encoded = label_encoder.fit_transform(y)

    # Membagi data menjadi train dan test (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

    # Menampilkan ukuran data train dan test
    print("Ukuran Train Set:", X_train.shape)
    print("Ukuran Test Set:", X_test.shape)

    # Melatih model Random Forest
    model = KNeighborsClassifier(n_neighbors=9)
    model.fit(X_train, y_train)

    # Memprediksi dengan data test
    y_pred = model.predict(X_test)

    # Menghitung akurasi model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Akurasi: {accuracy:.4f}")

    # Menampilkan laporan klasifikasizz
    print("Laporan Klasifikasi:")
    print(classification_report(y_test, y_pred))

    # Prediksi untuk input manual
    input_manual = pd.DataFrame([[5, 60, 80, 5]], columns=['Selisih', 'bars', 'kpi', 'Position'])

    # Prediksi dengan model yang sudah dilatih
    prediksi = model.predict(input_manual)

    # Mengembalikan hasil prediksi ke label yang dapat dimengerti
    prediksi_label = label_encoder.inverse_transform(prediksi)

    print(f"Prediksi untuk input manual: {prediksi_label[0]}")
    return prediksi_label[0]

In [ ]:

try:
    # Koneksi ke database
    connection = pymysql.connect(
        user="root",
        password="",
        host="localhost",
        port=3306,
        database="markovkaryawandb"
    )

    cursor = connection.cursor()

    # Data department dengan jumlah kekurangan pegawai

    # Misal markov_matrix dan markov_numpy sudah didefinisikan sebelumnya
    # Contoh dictionary input:
    depart_kurang = {"Operations Department" : 4, "Finance & Administration Department": 2}
    # quantity_kurang = [4,2]
    depart_tambah = {"Production Department":2, "Surabaya Branch Office Department":2,"Treasury Department":2}
    # quantity_tambah = [2,2,2]

    # Pastikan nama row/column ada di markov_matrix
    if all(k in markov_matrix.index for k in depart_kurang) and all(t in markov_matrix.index for t in depart_tambah):
        
        output = []  # Output akhir: list yang berisi list alokasi untuk tiap depart_kurang
        # Buat salinan kapasitas global untuk depart_tambah (akan dikurangi seiring alokasi)
        remaining_capacity = depart_tambah.copy()
        # Urutan key dari depart_tambah (penting agar output konsisten)
        depart_tambah_keys = list(depart_tambah.keys())
        
        # Loop tiap depart_kurang
        for index1, q1 in depart_kurang.items():
            new_probability = []
            
            # Ambil baris dari markov_matrix untuk depart_kurang
            search_array = markov_matrix.loc[index1].to_numpy()
            mc_p1 = np.linalg.matrix_power(markov_numpy, 1)
            hasil = np.dot(search_array, mc_p1)
            
            # Hitung probabilitas untuk tiap depart_tambah (sesuai urutan depart_tambah_keys)
            for key in depart_tambah_keys:
                col_index = markov_matrix.columns.get_loc(key)
                prob = hasil[col_index]
                new_probability.append(prob)
                print(f"Probabilitas menuju state '{key}': {prob}")
            print("---------------------------------------------------------------------")
            
            # Normalisasi probabilitas agar jumlahnya 1 (jika total > 0)
            total_prob = sum(new_probability)
            if total_prob > 0:
                norm_probability = [p / total_prob for p in new_probability]
            else:
                norm_probability = [1 / len(new_probability)] * len(new_probability)
            
            # Inisialisasi alokasi untuk baris ini, dalam bentuk list dengan panjang sama dengan jumlah depart_tambah
            allocation = [0] * len(depart_tambah_keys)
            items_to_allocate = q1
            
            # Lakukan alokasi item sebanyak q1 unit
            while items_to_allocate > 0:
                # Buat daftar kandidat (indeks) dari depart_tambah yang masih memiliki sisa kapasitas
                candidates = []
                candidate_weights = []
                for i, key in enumerate(depart_tambah_keys):
                    if remaining_capacity[key] > 0:
                        candidates.append(i)
                        candidate_weights.append(norm_probability[i])
                
                # Jika tidak ada kandidat tersisa, keluarkan pesan peringatan dan hentikan alokasi untuk baris ini
                if not candidates:
                    print("Tidak ada kapasitas tersisa untuk alokasi!")
                    break
                
                # Pilih secara acak salah satu kandidat menggunakan bobot
                chosen_idx = random.choices(candidates, weights=candidate_weights, k=1)[0]
                allocation[chosen_idx] += 1
                remaining_capacity[depart_tambah_keys[chosen_idx]] -= 1
                items_to_allocate -= 1
            
            output.append(allocation)
        
        print("Distribusi akhir:")
        print(output)
    else:
        print("error")

    output = []

    for name, q1 in depart_kurang.items():  # Menggunakan .items() agar bisa mengakses key dan value
        # Menggunakan parameterized query untuk menghindari SQL injection
        depart_id_query = "SELECT id FROM department WHERE name = %s"
        cursor.execute(depart_id_query, (name,))
        depart_id = cursor.fetchall()
        query = "SELECT id, full_name, position FROM employee WHERE department = %s"
        cursor.execute(query, (depart_id,))

        # Fetch semua data
        users = cursor.fetchall()

        # Menambahkan hasil ke output
        output.append({
            "department": name,
            "needed": q1,
            "employees": users
        })

    # Menutup koneksi setelah looping selesai
    cursor.close()
    connection.close()

    # Menampilkan hasil
    print(output)

except Exception as e:
    print({"message": str(e)})

Probabilitas menuju state 'Production Department': 0.027998228865271424
Probabilitas menuju state 'Surabaya Branch Office Department': 2.4752475247524726e-09
Probabilitas menuju state 'Treasury Department': 1.38267246570974e-05
---------------------------------------------------------------------
Probabilitas menuju state 'Production Department': 0.007033968650889921
Probabilitas menuju state 'Surabaya Branch Office Department': 2.2522522522522443e-09
Probabilitas menuju state 'Treasury Department': 0.004857225808298242
---------------------------------------------------------------------
Distribusi akhir:
[[2, 0, 2], [0, 2, 0]]
[{'department': 'Operations Department', 'needed': 4, 'employees': ()}, {'department': 'Finance & Administration Department', 'needed': 2, 'employees': ()}]
